In [0]:
data1=[(1,"Steve"),(2,"David"),(3,"John"),(4,"Shree"),(5,"Helen")]
data2=[(1,"SQL",90),(1,"PySpark",100),(2,"SQL",70),(2,"PySpark",60),(3,"SQL",30),(3,"PySpark",20),(4,"SQL",50),(4,"PySpark",50),(5,"SQL",45),(5,"PySpark",45)]

schema1=["Id","Name"]
schema2=["Id","Subject","Mark"]

df1=spark.createDataFrame(data1,schema1)
df2=spark.createDataFrame(data2,schema2)
display(df1)
display(df2)

Id,Name
1,Steve
2,David
3,John
4,Shree
5,Helen


Id,Subject,Mark
1,SQL,90
1,PySpark,100
2,SQL,70
2,PySpark,60
3,SQL,30
3,PySpark,20
4,SQL,50
4,PySpark,50
5,SQL,45
5,PySpark,45


In [0]:
from pyspark.sql.functions import *
df_join =df2.join(broadcast (df1),df1.Id==df2.Id,"left")

display(df_join)

Id,Subject,Mark,Id,Name
1,SQL,90,1,Steve
1,PySpark,100,1,Steve
2,SQL,70,2,David
2,PySpark,60,2,David
3,SQL,30,3,John
3,PySpark,20,3,John
4,SQL,50,4,Shree
4,PySpark,50,4,Shree
5,SQL,45,5,Helen
5,PySpark,45,5,Helen


In [0]:
df_join.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- BroadcastHashJoin [Id#6L], [Id#2L], LeftOuter, BuildRight, false, true
   :- Scan ExistingRDD[Id#6L,Subject#7,Mark#8L]
   +- Exchange SinglePartition, EXECUTOR_BROADCAST, [plan_id=957]
      +- Filter isnotnull(Id#2L)
         +- Scan ExistingRDD[Id#2L,Name#3]




In [0]:
df3=df_join.select(df1['Id'],df1['Name'],df2['Mark']).groupBy(col("Id"),col("Name")).agg(
    sum(col("Mark")/2).alias("total_marks")
)
display(df3)

Id,Name,total_marks
1,Steve,95.0
2,David,65.0
3,John,25.0
4,Shree,50.0
5,Helen,45.0


In [0]:
df3 = df3.withColumn("Grade",
                     when(df3.total_marks > 90, "Distinction")
                     .when(df3.total_marks > 60, "First class")
                     .when((df3.total_marks <= 50) & (df3.total_marks > 40), "Pass")
                     .otherwise("Fail"))   
                                
display(df3)                   

Id,Name,total_marks,Grade
1,Steve,95.0,Distinction
2,David,65.0,First class
3,John,25.0,Fail
4,Shree,50.0,Pass
5,Helen,45.0,Pass
